<p style="font-size:32px;text-align:center"> <b>Social network Graph Link Prediction - Facebook Challenge</b> </p>

In [1]:
#Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd#pandas to create small dataframes 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
# to install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm

# 1. Reading Data

In [2]:
if os.path.isfile('data/after_eda/train_pos_after_eda.csv'):
    train_graph=nx.read_edgelist('data/after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
    print(nx.info(train_graph))
else:
    print("please run the FB_EDA.ipynb or download the files from drive")

DiGraph with 1780722 nodes and 7550015 edges


# 2. Similarity measures

## 2.1 Jaccard Distance:
http://www.statisticshowto.com/jaccard-index/

\begin{equation}
j = \frac{|X\cap Y|}{|X \cup Y|} 
\end{equation}

In [3]:
#for followees
def jaccard_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (len(set(train_graph.successors(a)).union(set(train_graph.successors(b)))))
    except:
        return 0
    return sim

In [4]:
#one test case
print(jaccard_for_followees(273084,1505602))

0.0


In [5]:
#node 1635354 not in graph 
print(jaccard_for_followees(273084,1505602))

0.0


In [6]:
#for followers
def jaccard_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a))) == 0  | len(set(g.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                 (len(set(train_graph.predecessors(a)).union(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

In [7]:
print(jaccard_for_followers(273084,470294))

0


In [8]:
#node 1635354 not in graph 
print(jaccard_for_followees(669354,1635354))

0


## 2.2 Cosine distance

\begin{equation}
CosineDistance = \frac{|X\cap Y|}{|X|\cdot|Y|} 
\end{equation}

In [9]:
#for followees
def cosine_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (math.sqrt(len(set(train_graph.successors(a)))*len((set(train_graph.successors(b))))))
        return sim
    except:
        return 0

In [10]:
print(cosine_for_followees(273084,1505602))

0.0


In [11]:
print(cosine_for_followees(273084,1635354))

0


In [12]:
def cosine_for_followers(a,b):
    try:
        
        if len(set(train_graph.predecessors(a))) == 0  | len(set(train_graph.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                     (math.sqrt(len(set(train_graph.predecessors(a))))*(len(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

In [13]:
print(cosine_for_followers(2,470294))

0.02886751345948129


In [14]:
print(cosine_for_followers(669354,1635354))

0


## 3. Ranking Measures

https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html

PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links.

<img src='PageRanks-Example.jpg'/>

Mathematical PageRanks for a simple network, expressed as percentages. (Google uses a logarithmic scale.) Page C has a higher PageRank than Page E, even though there are fewer links to C; the one link to C comes from an important page and hence is of high value. If web surfers who start on a random page have an 85% likelihood of choosing a random link from the page they are currently visiting, and a 15% likelihood of jumping to a page chosen at random from the entire web, they will reach Page E 8.1% of the time. <b>(The 15% likelihood of jumping to an arbitrary page corresponds to a damping factor of 85%.) Without damping, all web surfers would eventually end up on Pages A, B, or C, and all other pages would have PageRank zero. In the presence of damping, Page A effectively links to all pages in the web, even though it has no outgoing links of its own.</b>

## 3.1 Page Ranking

https://en.wikipedia.org/wiki/PageRank


# 4. Other Graph Features

## 4.1 Shortest path:

Getting Shortest path between twoo nodes, if nodes have direct path i.e directly connected then we are removing that edge and calculating path. 

In [15]:
#if has direct edge then deleting that edge and calculating shortest path
def compute_shortest_path_length(a,b):
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p= nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p= nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1

In [16]:
#testing
compute_shortest_path_length(77697, 826021)

10

In [17]:
#testing
compute_shortest_path_length(669354,1635354)

-1

## 4.2 Checking for same community

In [18]:
#getting weekly connected edges from graph 
wcc=list(nx.weakly_connected_components(train_graph))
def belongs_to_same_wcc(a,b):
    index = []
    if train_graph.has_edge(b,a):
        return 1
    if train_graph.has_edge(a,b):
            for i in wcc:
                if a in i:
                    index= i
                    break
            if (b in index):
                train_graph.remove_edge(a,b)
                if compute_shortest_path_length(a,b)==-1:
                    train_graph.add_edge(a,b)
                    return 0
                else:
                    train_graph.add_edge(a,b)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if a in i:
                    index= i
                    break
            if(b in index):
                return 1
            else:
                return 0

In [19]:
belongs_to_same_wcc(861, 1659750)

0

In [20]:
belongs_to_same_wcc(669354,1635354)

0

## 4.3 Adamic/Adar Index:
Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

In [21]:
#adar index
def calc_adar_in(a,b):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(train_graph.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

In [22]:
calc_adar_in(1,189226)

0

In [26]:
calc_adar_in(669354,1635354)

0

## 4.4 Is persion was following back:

In [27]:
def follows_back(a,b):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0

In [28]:
follows_back(1,189226)

1

In [29]:
follows_back(669354,1635354)

0

## 4.5 Katz Centrality:
https://en.wikipedia.org/wiki/Katz_centrality

https://www.geeksforgeeks.org/katz-centrality-centrality-measure/
 Katz centrality computes the centrality for a node 
    based on the centrality of its neighbors. It is a 
    generalization of the eigenvector centrality. The
    Katz centrality for node `i` is
 
$$x_i = \alpha \sum_{j} A_{ij} x_j + \beta,$$
where `A` is the adjacency matrix of the graph G 
with eigenvalues $$\lambda$$.

The parameter $$\beta$$ controls the initial centrality and 

$$\alpha < \frac{1}{\lambda_{max}}.$$

In [30]:
if not os.path.isfile('data/fea_sample/katz.p'):
    katz = nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
    pickle.dump(katz,open('data/fea_sample/katz.p','wb'))
else:
    katz = pickle.load(open('data/fea_sample/katz.p','rb'))

In [31]:
print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
print('mean',float(sum(katz.values())) / len(katz))

min 0.0007313532484065916
max 0.003394554981699122
mean 0.0007483800935562018


In [32]:
mean_katz = float(sum(katz.values())) / len(katz)
print(mean_katz)

0.0007483800935562018


## 4.6 Hits Score
The HITS algorithm computes two numbers for a node. Authorities estimates the node value based on the incoming links. Hubs estimates the node value based on outgoing links.

https://en.wikipedia.org/wiki/HITS_algorithm

In [33]:
if not os.path.isfile('data/fea_sample/hits.p'):
    hits = nx.hits(train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    pickle.dump(hits,open('data/fea_sample/hits.p','wb'))
else:
    hits = pickle.load(open('data/fea_sample/hits.p','rb'))

In [34]:
print('min',hits[0][min(hits[0], key=hits[0].get)])
print('max',hits[0][max(hits[0], key=hits[0].get)])
print('mean',float(sum(hits[0].values())) / len(hits[0]))

min 0.0
max 0.004868653378780953
mean 5.615699699344123e-07


# 5. Featurization

## 5. 1 Reading a sample of Data from both train and test

In [35]:
! gdown --id 1lcxzVZ0-MkPmoH3lS35Q8rRfrecKSXb1
! gdown --id 1_KN7S8zfHdrkRjRYOEtBxBVq8JrGxPXD

zsh:1: command not found: gdown
zsh:1: command not found: gdown


In [40]:
import random
if os.path.isfile('data/after_eda/train_after_eda.csv'):
    filename = "train_after_eda.csv"
    # you uncomment this line, if you dont know the lentgh of the file name
    # here we have hardcoded the number of lines as 15100030
    # n_train = sum(1 for line in open(filename)) #number of records in file (excludes header)
    n_train =  15100028
    s = 100000 #desired sample size
    skip_train = sorted(random.sample(range(1,n_train+1),n_train-s))
    #https://stackoverflow.com/a/22259008/4084039

In [39]:
if os.path.isfile('data/after_eda/test_after_eda.csv'):
    filename = "test_after_eda.csv"
    # you uncomment this line, if you dont know the lentgh of the file name
    # here we have hardcoded the number of lines as 3775008
    # n_test = sum(1 for line in open(filename)) #number of records in file (excludes header)
    n_test = 3775006
    s = 50000 #desired sample size
    skip_test = sorted(random.sample(range(1,n_test+1),n_test-s))
    #https://stackoverflow.com/a/22259008/4084039

In [41]:
print("Number of rows in the train data file:", n_train)
print("Number of rows we are going to elimiate in train data are",len(skip_train))
print("Number of rows in the test data file:", n_test)
print("Number of rows we are going to elimiate in test data are",len(skip_test))

Number of rows in the train data file: 15100028
Number of rows we are going to elimiate in train data are 15000028
Number of rows in the test data file: 3775006
Number of rows we are going to elimiate in test data are 3725006


In [42]:
#https://drive.google.com/file/d/19mviN_yeJIfakb4kU5NfKdQlOQtaQ-kH/view?usp=sharing
!gdown --id 19mviN_yeJIfakb4kU5NfKdQlOQtaQ-kH

zsh:1: command not found: gdown


In [43]:
#https://drive.google.com/file/d/1H6qybuXr8i_USWu3k3ulXEOurc-SElUh/view?usp=sharing
!gdown --id 1H6qybuXr8i_USWu3k3ulXEOurc-SElUh

zsh:1: command not found: gdown


In [47]:
df_final_train = pd.read_csv('data/after_eda/train_after_eda.csv', skiprows=skip_train, names=['source_node', 'destination_node'])
df_final_train['indicator_link'] = pd.read_csv('data/train_y.csv', skiprows=skip_train, names=['indicator_link'])
print("Our train matrix size ",df_final_train.shape)
df_final_train.head(2)

Our train matrix size  (100002, 3)


,source_node,destination_node,indicator_link
0,273084,1505602,1
1,690085,1100574,1


In [48]:
df_final_test = pd.read_csv('data/after_eda/test_after_eda.csv', skiprows=skip_train, names=['source_node', 'destination_node'])
df_final_test['indicator_link'] = pd.read_csv('data/test_y.csv', skiprows=skip_train, names=['indicator_link'])
print("Our train matrix size ",df_final_test.shape)
df_final_test.head(2)

Our train matrix size  (25156, 3)


,source_node,destination_node,indicator_link
0,848424,784690,1
1,1288618,816620,1


## 5.2 Adding a set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>jaccard_followers</li>
<li>jaccard_followees</li>
<li>cosine_followers</li>
<li>cosine_followees</li>
<li>num_followers_s</li>
<li>num_followees_s</li>
<li>num_followers_d</li>
<li>num_followees_d</li>
<li>inter_followers</li>
<li>inter_followees</li>
</ol>

In [49]:
def compute_features_stage1(df_final):
    #calculating no of followers followees for source and destination
    #calculating intersection of followers and followees for source and destination
    num_followers_s=[]
    num_followees_s=[]
    num_followers_d=[]
    num_followees_d=[]
    inter_followers=[]
    inter_followees=[]
    for i,row in df_final.iterrows():
        try:
            s1=set(train_graph.predecessors(row['source_node']))
            s2=set(train_graph.successors(row['source_node']))
        except:
            s1 = set()
            s2 = set()
        try:
            d1=set(train_graph.predecessors(row['destination_node']))
            d2=set(train_graph.successors(row['destination_node']))
        except:
            d1 = set()
            d2 = set()
        num_followers_s.append(len(s1))
        num_followees_s.append(len(s2))

        num_followers_d.append(len(d1))
        num_followees_d.append(len(d2))

        inter_followers.append(len(s1.intersection(d1)))
        inter_followees.append(len(s2.intersection(d2)))
    
    return  num_followers_s,num_followees_s,num_followers_d,num_followees_d,inter_followers,inter_followees

In [51]:

df_final_train['num_followers_s'], df_final_train['num_followers_d'], \
df_final_train['num_followees_s'], df_final_train['num_followees_d'], \
df_final_train['inter_followers'], df_final_train['inter_followees']= compute_features_stage1(df_final_train)
    
df_final_test['num_followers_s'], df_final_test['num_followers_d'], \
df_final_test['num_followees_s'], df_final_test['num_followees_d'], \
df_final_test['inter_followers'], df_final_test['inter_followees']= compute_features_stage1(df_final_test)


In [52]:
df_final_train.head()


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees
0,273084,1505602,1,11,15,6,8,0,0
1,690085,1100574,1,1,1,2,2,0,0
2,1430336,300365,1,32,46,5,0,0,0
3,471874,1256320,1,123,202,109,176,67,134
4,267343,1183878,1,1,1,3,4,0,0


In [53]:
a=df_final_train['num_followers_s'].values
b=df_final_train['num_followers_d'].values
for x,y in (zip(a,b)):
  if x==0:
    if y!=0:
      print('i')


i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i


i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i


In [54]:
np.count_nonzero(a)

89623

In [55]:
np.count_nonzero(b)

89722

In [ ]:
# ! gdown --id 1fDJptlCFEWNV5UNGPc4geTykgFI3PDCV

In [ ]:
# df_final_train = read_hdf('storage_sample_stage4.h5', 'train_df',mode='r')
# df_final_test = read_hdf('storage_sample_stage4.h5', 'test_df',mode='r')

In [ ]:
# df_final_train.tail()

In [ ]:
# df_final_train_new=df_final_train.drop(['num_followers_s',	'num_followees_s',	'num_followees_d'	,'inter_followers',	'inter_followees'],axis=1)

In [ ]:
# df_final_train['num_followers_d']= compute_features_stage1(df_final_train)

In [ ]:
# df_final_train.tail()

In [ ]:
# for val in df_final_train_new['num_followers_s'].values:
#   if(val>0):
#     print(val)

In [ ]:
# https://drive.google.com/file/d/10qJ04GRcaDxc16gmJXb8rpGPmlyys7E2/view?usp=sharing
! gdown --id 10qJ04GRcaDxc16gmJXb8rpGPmlyys7E2


## 5.3 Adding new set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>adar index</li>
<li>is following back</li>
<li>belongs to same weakly connect components</li>
<li>shortest path between source and destination</li>
</ol>

In [56]:
df_final_train['adar_index'] = df_final_train.apply(lambda row: calc_adar_in(row['source_node'],row['destination_node']),axis=1)
#mapping adar index on test
df_final_test['adar_index'] = df_final_test.apply(lambda row: calc_adar_in(row['source_node'],row['destination_node']),axis=1)

#--------------------------------------------------------------------------------------------------------
#mapping followback or not on train
df_final_train['follows_back'] = df_final_train.apply(lambda row: follows_back(row['source_node'],row['destination_node']),axis=1)

#mapping followback or not on test
df_final_test['follows_back'] = df_final_test.apply(lambda row: follows_back(row['source_node'],row['destination_node']),axis=1)

#--------------------------------------------------------------------------------------------------------
#mapping same component of wcc or not on train
df_final_train['same_comp'] = df_final_train.apply(lambda row: belongs_to_same_wcc(row['source_node'],row['destination_node']),axis=1)

##mapping same component of wcc or not on train
df_final_test['same_comp'] = df_final_test.apply(lambda row: belongs_to_same_wcc(row['source_node'],row['destination_node']),axis=1)
    
#--------------------------------------------------------------------------------------------------------
#mapping shortest path on train 
df_final_train['shortest_path'] = df_final_train.apply(lambda row: compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)
#mapping shortest path on test
df_final_test['shortest_path'] = df_final_test.apply(lambda row: compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)

In [57]:
df_final_train.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,follows_back,same_comp,shortest_path
0,273084,1505602,1,11,15,6,8,0,0,0.000000,0,1,4
1,690085,1100574,1,1,1,2,2,0,0,0.000000,1,1,-1
2,1430336,300365,1,32,46,5,0,0,0,0.000000,0,1,3
3,471874,1256320,1,123,202,109,176,67,134,64.366813,1,1,2
4,267343,1183878,1,1,1,3,4,0,0,0.000000,1,1,-1


## 5.4 Adding new set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>Weight Features
    <ul>
        <li>weight of incoming edges</li>
        <li>weight of outgoing edges</li>
        <li>weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges * weight of outgoing edges</li>
        <li>2*weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges + 2*weight of outgoing edges</li>
    </ul>
</li>
<li>Page Ranking of source</li>
<li>Page Ranking of dest</li>
<li>katz of source</li>
<li>katz of dest</li>
<li>hubs of source</li>
<li>hubs of dest</li>
<li>authorities_s of source</li>
<li>authorities_s of dest</li>
</ol>

#### Weight Features

In order to determine the similarity of nodes, an edge weight value was calculated between nodes. Edge weight decreases as the neighbor count goes up. Intuitively, consider one million people following a celebrity on a social network then chances are most of them never met each other or the celebrity. On the other hand, if a user has 30 contacts in his/her social network, the chances are higher that many of them know each other. 
`credit` - Graph-based Features for Supervised Link Prediction
William Cukierski, Benjamin Hamner, Bo Yang

\begin{equation}
W = \frac{1}{\sqrt{1+|X|}}
\end{equation}

it is directed graph so calculated Weighted in and Weighted out differently

In [58]:
#weight for source and destination of each link
Weight_in = {}
Weight_out = {}
for i in  tqdm(train_graph.nodes()):
    s1=set(train_graph.predecessors(i))
    w_in = 1.0/(np.sqrt(1+len(s1)))
    Weight_in[i]=w_in
    
    s2=set(train_graph.successors(i))
    w_out = 1.0/(np.sqrt(1+len(s2)))
    Weight_out[i]=w_out
    
#for imputing with mean
mean_weight_in = np.mean(list(Weight_in.values()))
mean_weight_out = np.mean(list(Weight_out.values()))

100%|██████████████████████████████| 1780722/1780722 [00:48<00:00, 36758.18it/s]


In [60]:

#mapping to pandas train
df_final_train['weight_in'] = df_final_train.destination_node.apply(lambda x: Weight_in.get(x,mean_weight_in))
df_final_train['weight_out'] = df_final_train.source_node.apply(lambda x: Weight_out.get(x,mean_weight_out))

#mapping to pandas test
df_final_test['weight_in'] = df_final_test.destination_node.apply(lambda x: Weight_in.get(x,mean_weight_in))
df_final_test['weight_out'] = df_final_test.source_node.apply(lambda x: Weight_out.get(x,mean_weight_out))


#some features engineerings on the in and out weights
df_final_train['weight_f1'] = df_final_train.weight_in + df_final_train.weight_out
df_final_train['weight_f2'] = df_final_train.weight_in * df_final_train.weight_out
df_final_train['weight_f3'] = (2*df_final_train.weight_in + 1*df_final_train.weight_out)
df_final_train['weight_f4'] = (1*df_final_train.weight_in + 2*df_final_train.weight_out)

#some features engineerings on the in and out weights
df_final_test['weight_f1'] = df_final_test.weight_in + df_final_test.weight_out
df_final_test['weight_f2'] = df_final_test.weight_in * df_final_test.weight_out
df_final_test['weight_f3'] = (2*df_final_test.weight_in + 1*df_final_test.weight_out)
df_final_test['weight_f4'] = (1*df_final_test.weight_in + 2*df_final_test.weight_out)

In [ ]:
pr = nx.pagerank(train_graph,alpha=0.85)
hits = nx.hits(train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)


In [67]:
mean_pr = float(sum(pr.values())) / len(pr)
mean_katz = float(sum(katz.values())) / len(katz)
#page rank for source and destination in Train and Test
#if anything not there in train graph then adding mean page rank 
df_final_train['page_rank_s'] = df_final_train.source_node.apply(lambda x:pr.get(x,mean_pr))
df_final_train['page_rank_d'] = df_final_train.destination_node.apply(lambda x:pr.get(x,mean_pr))

df_final_test['page_rank_s'] = df_final_test.source_node.apply(lambda x:pr.get(x,mean_pr))
df_final_test['page_rank_d'] = df_final_test.destination_node.apply(lambda x:pr.get(x,mean_pr))
    #================================================================================

#Katz centrality score for source and destination in Train and test
#if anything not there in train graph then adding mean katz score
df_final_train['katz_s'] = df_final_train.source_node.apply(lambda x: katz.get(x,mean_katz))
df_final_train['katz_d'] = df_final_train.destination_node.apply(lambda x: katz.get(x,mean_katz))

df_final_test['katz_s'] = df_final_test.source_node.apply(lambda x: katz.get(x,mean_katz))
df_final_test['katz_d'] = df_final_test.destination_node.apply(lambda x: katz.get(x,mean_katz))
#================================================================================

#Hits algorithm score for source and destination in Train and test
#if anything not there in train graph then adding 0
df_final_train['hubs_s'] = df_final_train.source_node.apply(lambda x: hits[0].get(x,0))
df_final_train['hubs_d'] = df_final_train.destination_node.apply(lambda x: hits[0].get(x,0))

df_final_test['hubs_s'] = df_final_test.source_node.apply(lambda x: hits[0].get(x,0))
df_final_test['hubs_d'] = df_final_test.destination_node.apply(lambda x: hits[0].get(x,0))
    #================================================================================

#Hits algorithm score for source and destination in Train and Test
#if anything not there in train graph then adding 0
df_final_train['authorities_s'] = df_final_train.source_node.apply(lambda x: hits[1].get(x,0))
df_final_train['authorities_d'] = df_final_train.destination_node.apply(lambda x: hits[1].get(x,0))

df_final_test['authorities_s'] = df_final_test.source_node.apply(lambda x: hits[1].get(x,0))
df_final_test['authorities_d'] = df_final_test.destination_node.apply(lambda x: hits[1].get(x,0))


In [68]:
df_final_train.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,...,weight_f3,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d
0,273084,1505602,1,11,15,6,8,0,0,0.000000,...,1.005929,0.877964,2.046156e-06,3.468618e-07,0.000773,0.000756,1.943132e-13,1.941005e-13,9.226317e-16,2.231877e-15
1,690085,1100574,1,1,1,2,2,0,0,0.000000,...,2.414214,2.707107,4.050978e-07,2.857641e-07,0.000735,0.000739,-0.000000e+00,-2.150947e-25,-1.498366e-23,-7.543153e-23
2,1430336,300365,1,32,46,5,0,0,0,0.000000,...,0.962362,0.699978,1.844829e-06,3.961420e-07,0.000866,0.000751,2.154703e-11,-0.000000e+00,2.591302e-12,1.544850e-13
3,471874,1256320,1,123,202,109,176,67,134,64.366813,...,0.260879,0.235719,6.039810e-07,5.190366e-07,0.001912,0.001802,4.260184e-03,3.996260e-03,3.011682e-03,2.724188e-03
4,267343,1183878,1,1,1,3,4,0,0,0.000000,...,2.154701,2.577350,2.857641e-07,4.448756e-07,0.000735,0.000743,-0.000000e+00,9.795056e-16,6.888066e-18,4.230035e-19


## 5.5 Adding new set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>SVD features for both source and destination</li>
</ol>

In [69]:
def svd(x, S):
    try:
        z = sadj_dict[x]
        return S[z]
    except:
        return [0,0,0,0,0,0]

In [70]:
#for svd features to get feature vector creating a dict node val and inedx in svd vector
sadj_col = sorted(train_graph.nodes())
sadj_dict = { val:idx for idx,val in enumerate(sadj_col)}

In [71]:
Adj = nx.adjacency_matrix(train_graph,nodelist=sorted(train_graph.nodes())).asfptype()

In [72]:
U, s, V = svds(Adj, k = 6)
print('Adjacency matrix Shape',Adj.shape)
print('U Shape',U.shape)
print('V Shape',V.shape)
print('s Shape',s.shape)

Adjacency matrix Shape (1780722, 1780722)
U Shape (1780722, 6)
V Shape (6, 1780722)
s Shape (6,)


In [74]:

df_final_train[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
df_final_train.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    
df_final_train[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
df_final_train.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)
#===================================================================================================

df_final_train[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',]] = \
df_final_train.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

df_final_train[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
df_final_train.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)
#===================================================================================================

df_final_test[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
df_final_test.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)

df_final_test[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
df_final_test.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)

#===================================================================================================

df_final_test[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',]] = \
df_final_test.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

df_final_test[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
df_final_test.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)
#===================================================================================================


In [75]:
source = np.array(df_final_train["num_followers_s"])
destination = np.array(df_final_train["num_followers_d"])
preferential_followers = []
for i in range(len(source)):
    preferential_followers.append(source[i]*destination[i])
df_final_train["followers_preferential_attach"]  = preferential_followers
df_final_train.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,...,svd_v_s_4,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,followers_preferential_attach
0,273084,1505602,1,11,15,6,8,0,0,0.000000,...,1.545080e-13,-8.108435e-13,1.719701e-14,1.355368e-12,4.675306e-13,1.128590e-06,6.616657e-14,-9.771039e-13,4.160012e-14,165
1,690085,1100574,1,1,1,2,2,0,0,0.000000,...,-2.612821e-22,-9.606660e-24,-9.918692e-23,2.379641e-19,1.953984e-17,7.685073e-18,5.145229e-19,-1.700560e-18,7.326186e-22,1
2,1430336,300365,1,32,46,5,0,0,0,0.000000,...,3.562464e-10,-3.159209e-09,4.829948e-11,6.679381e-12,4.168967e-11,7.840550e-11,7.360802e-12,-2.494395e-11,2.879458e-12,1472
3,471874,1256320,1,123,202,109,176,67,134,64.366813,...,-4.438838e-09,1.990834e-12,5.613496e-02,-8.590055e-14,-5.213415e-13,-2.495579e-10,-2.944627e-09,1.789286e-12,5.077634e-02,24846
4,267343,1183878,1,1,1,3,4,0,0,0.000000,...,4.016938e-17,-1.274182e-16,1.283867e-16,8.885049e-14,2.207404e-15,2.692761e-14,2.423655e-15,-3.798985e-14,7.899208e-18,1


In [76]:
source = np.array(df_final_train["num_followees_s"])
destination = np.array(df_final_train["num_followees_s"])
preferential_followees = []
for i in range(len(source)):
    preferential_followees.append(source[i]*destination[i])
df_final_train["followees_preferential_attach"]  = preferential_followees
df_final_train.head()


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,...,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,followers_preferential_attach,followees_preferential_attach
0,273084,1505602,1,11,15,6,8,0,0,0.000000,...,-8.108435e-13,1.719701e-14,1.355368e-12,4.675306e-13,1.128590e-06,6.616657e-14,-9.771039e-13,4.160012e-14,165,36
1,690085,1100574,1,1,1,2,2,0,0,0.000000,...,-9.606660e-24,-9.918692e-23,2.379641e-19,1.953984e-17,7.685073e-18,5.145229e-19,-1.700560e-18,7.326186e-22,1,4
2,1430336,300365,1,32,46,5,0,0,0,0.000000,...,-3.159209e-09,4.829948e-11,6.679381e-12,4.168967e-11,7.840550e-11,7.360802e-12,-2.494395e-11,2.879458e-12,1472,25
3,471874,1256320,1,123,202,109,176,67,134,64.366813,...,1.990834e-12,5.613496e-02,-8.590055e-14,-5.213415e-13,-2.495579e-10,-2.944627e-09,1.789286e-12,5.077634e-02,24846,11881
4,267343,1183878,1,1,1,3,4,0,0,0.000000,...,-1.274182e-16,1.283867e-16,8.885049e-14,2.207404e-15,2.692761e-14,2.423655e-15,-3.798985e-14,7.899208e-18,1,9


In [77]:
source = np.array(df_final_test["num_followers_s"])
destination = np.array(df_final_test["num_followers_d"])
preferential_followers_test = []
for i in range(len(source)):
    preferential_followers_test.append(source[i]*destination[i])
df_final_test["followers_preferential_attach"]  = preferential_followers_test
df_final_test.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,...,svd_v_s_4,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,followers_preferential_attach
0,848424,784690,1,6,6,14,9,1,0,0.000000,...,2.701538e-12,-4.341443e-13,5.535490e-14,9.994076e-10,5.791909e-10,3.512363e-07,2.486660e-09,-2.771007e-09,1.727693e-12,36
1,1288618,816620,1,24,31,12,22,4,10,11.923852,...,8.327162e-10,-3.605099e-11,1.313545e-10,9.083281e-10,6.860622e-10,2.349945e-11,4.769777e-10,-1.302439e-09,5.961292e-11,744
2,417653,942104,1,40,30,11,15,1,0,0.000000,...,7.432161e-13,-1.129036e-11,1.104040e-13,1.449897e-11,2.901599e-11,1.997404e-10,4.238304e-12,-2.970304e-11,2.816912e-10,1200
3,971595,646855,1,8,4,4,3,2,0,0.000000,...,4.511475e-12,-1.347928e-12,8.104501e-14,8.800713e-13,1.547198e-14,4.328435e-08,5.001930e-14,-2.323153e-13,1.351607e-14,32
4,270084,1443419,1,48,65,3,6,1,3,2.331492,...,7.685401e-10,-5.911465e-10,3.324514e-13,7.087074e-13,1.648484e-12,5.121786e-12,7.230008e-13,-8.446612e-13,7.961847e-16,3120


In [78]:
source = np.array(df_final_test["num_followees_s"])
destination = np.array(df_final_test["num_followees_s"])
preferential_followees_test= []
for i in range(len(source)):
    preferential_followees_test.append(source[i]*destination[i])
df_final_test["followees_preferential_attach"]  = preferential_followees_test
df_final_test.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,...,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,followers_preferential_attach,followees_preferential_attach
0,848424,784690,1,6,6,14,9,1,0,0.000000,...,-4.341443e-13,5.535490e-14,9.994076e-10,5.791909e-10,3.512363e-07,2.486660e-09,-2.771007e-09,1.727693e-12,36,196
1,1288618,816620,1,24,31,12,22,4,10,11.923852,...,-3.605099e-11,1.313545e-10,9.083281e-10,6.860622e-10,2.349945e-11,4.769777e-10,-1.302439e-09,5.961292e-11,744,144
2,417653,942104,1,40,30,11,15,1,0,0.000000,...,-1.129036e-11,1.104040e-13,1.449897e-11,2.901599e-11,1.997404e-10,4.238304e-12,-2.970304e-11,2.816912e-10,1200,121
3,971595,646855,1,8,4,4,3,2,0,0.000000,...,-1.347928e-12,8.104501e-14,8.800713e-13,1.547198e-14,4.328435e-08,5.001930e-14,-2.323153e-13,1.351607e-14,32,16
4,270084,1443419,1,48,65,3,6,1,3,2.331492,...,-5.911465e-10,3.324514e-13,7.087074e-13,1.648484e-12,5.121786e-12,7.230008e-13,-8.446612e-13,7.961847e-16,3120,9


In [79]:
svd_dot_U =[]
range_value = len(np.array(df_final_train["svd_u_d_1"]))
for i in range(range_value):
    u_s = []
    u_d = []
    u_s.append(np.array(df_final_train["svd_u_s_1"][i]))
    u_s.append(np.array(df_final_train["svd_u_s_2"][i]))
    u_s.append(np.array(df_final_train["svd_u_s_3"][i]))
    u_s.append(np.array(df_final_train["svd_u_s_4"][i]))
    u_s.append(np.array(df_final_train["svd_u_s_5"][i]))
    u_s.append(np.array(df_final_train["svd_u_s_6"][i]))

    u_d.append(np.array(df_final_train["svd_u_d_1"][i]))
    u_d.append(np.array(df_final_train["svd_u_d_2"][i]))
    u_d.append(np.array(df_final_train["svd_u_d_3"][i]))
    u_d.append(np.array(df_final_train["svd_u_d_4"][i]))
    u_d.append(np.array(df_final_train["svd_u_d_5"][i]))
    u_d.append(np.array(df_final_train["svd_u_d_6"][i]))

    svd_dot_U.append(np.dot(u_s,u_d))

In [80]:
df_final_train["svd_dot_U"] = svd_dot_U
df_final_train.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,...,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,followers_preferential_attach,followees_preferential_attach,svd_dot_U
0,273084,1505602,1,11,15,6,8,0,0,0.000000,...,1.719701e-14,1.355368e-12,4.675306e-13,1.128590e-06,6.616657e-14,-9.771039e-13,4.160012e-14,165,36,1.114866e-11
1,690085,1100574,1,1,1,2,2,0,0,0.000000,...,-9.918692e-23,2.379641e-19,1.953984e-17,7.685073e-18,5.145229e-19,-1.700560e-18,7.326186e-22,1,4,-2.045047e-38
2,1430336,300365,1,32,46,5,0,0,0,0.000000,...,4.829948e-11,6.679381e-12,4.168967e-11,7.840550e-11,7.360802e-12,-2.494395e-11,2.879458e-12,1472,25,3.576731e-29
3,471874,1256320,1,123,202,109,176,67,134,64.366813,...,5.613496e-02,-8.590055e-14,-5.213415e-13,-2.495579e-10,-2.944627e-09,1.789286e-12,5.077634e-02,24846,11881,5.794851e-03
4,267343,1183878,1,1,1,3,4,0,0,0.000000,...,1.283867e-16,8.885049e-14,2.207404e-15,2.692761e-14,2.423655e-15,-3.798985e-14,7.899208e-18,1,9,6.965232e-32


In [81]:
svd_dot_V = []
range_value = len(np.array(df_final_train["svd_v_d_1"]))
for i in range(range_value):
    v_s = []
    v_d = []
    v_s.append(np.array(df_final_train["svd_v_s_1"][i]))
    v_s.append(np.array(df_final_train["svd_v_s_2"][i]))
    v_s.append(np.array(df_final_train["svd_v_s_3"][i]))
    v_s.append(np.array(df_final_train["svd_v_s_4"][i]))
    v_s.append(np.array(df_final_train["svd_v_s_5"][i]))
    v_s.append(np.array(df_final_train["svd_v_s_6"][i]))

    v_d.append(np.array(df_final_train["svd_v_d_1"][i]))
    v_d.append(np.array(df_final_train["svd_v_d_2"][i]))
    v_d.append(np.array(df_final_train["svd_v_d_3"][i]))
    v_d.append(np.array(df_final_train["svd_v_d_4"][i]))
    v_d.append(np.array(df_final_train["svd_v_d_5"][i]))
    v_d.append(np.array(df_final_train["svd_v_d_6"][i]))

    svd_dot_V.append(np.dot(v_s,v_d))

In [82]:
df_final_train["svd_dot_V"] = svd_dot_V
df_final_train.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,...,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,followers_preferential_attach,followees_preferential_attach,svd_dot_U,svd_dot_V
0,273084,1505602,1,11,15,6,8,0,0,0.000000,...,1.355368e-12,4.675306e-13,1.128590e-06,6.616657e-14,-9.771039e-13,4.160012e-14,165,36,1.114866e-11,2.238792e-12
1,690085,1100574,1,1,1,2,2,0,0,0.000000,...,2.379641e-19,1.953984e-17,7.685073e-18,5.145229e-19,-1.700560e-18,7.326186e-22,1,4,-2.045047e-38,1.887801e-41
2,1430336,300365,1,32,46,5,0,0,0,0.000000,...,6.679381e-12,4.168967e-11,7.840550e-11,7.360802e-12,-2.494395e-11,2.879458e-12,1472,25,3.576731e-29,4.904049e-19
3,471874,1256320,1,123,202,109,176,67,134,64.366813,...,-8.590055e-14,-5.213415e-13,-2.495579e-10,-2.944627e-09,1.789286e-12,5.077634e-02,24846,11881,5.794851e-03,2.850328e-03
4,267343,1183878,1,1,1,3,4,0,0,0.000000,...,8.885049e-14,2.207404e-15,2.692761e-14,2.423655e-15,-3.798985e-14,7.899208e-18,1,9,6.965232e-32,2.926482e-28


In [83]:
svd_dot_U =[]
range_value = len(np.array(df_final_test["svd_u_d_1"]))
for i in range(range_value):
    u_s = []
    u_d = []
    u_s.append(np.array(df_final_test["svd_u_s_1"][i]))
    u_s.append(np.array(df_final_test["svd_u_s_2"][i]))
    u_s.append(np.array(df_final_test["svd_u_s_3"][i]))
    u_s.append(np.array(df_final_test["svd_u_s_4"][i]))
    u_s.append(np.array(df_final_test["svd_u_s_5"][i]))
    u_s.append(np.array(df_final_test["svd_u_s_6"][i]))

    u_d.append(np.array(df_final_test["svd_u_d_1"][i]))
    u_d.append(np.array(df_final_test["svd_u_d_2"][i]))
    u_d.append(np.array(df_final_test["svd_u_d_3"][i]))
    u_d.append(np.array(df_final_test["svd_u_d_4"][i]))
    u_d.append(np.array(df_final_test["svd_u_d_5"][i]))
    u_d.append(np.array(df_final_test["svd_u_d_6"][i]))

    svd_dot_U.append(np.dot(u_s,u_d))

In [84]:
df_final_test["svd_dot_U"] = svd_dot_U
df_final_test.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,...,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,followers_preferential_attach,followees_preferential_attach,svd_dot_U
0,848424,784690,1,6,6,14,9,1,0,0.000000,...,5.535490e-14,9.994076e-10,5.791909e-10,3.512363e-07,2.486660e-09,-2.771007e-09,1.727693e-12,36,196,8.424654e-20
1,1288618,816620,1,24,31,12,22,4,10,11.923852,...,1.313545e-10,9.083281e-10,6.860622e-10,2.349945e-11,4.769777e-10,-1.302439e-09,5.961292e-11,744,144,8.289328e-17
2,417653,942104,1,40,30,11,15,1,0,0.000000,...,1.104040e-13,1.449897e-11,2.901599e-11,1.997404e-10,4.238304e-12,-2.970304e-11,2.816912e-10,1200,121,4.247511e-20
3,971595,646855,1,8,4,4,3,2,0,0.000000,...,8.104501e-14,8.800713e-13,1.547198e-14,4.328435e-08,5.001930e-14,-2.323153e-13,1.351607e-14,32,16,1.045200e-13
4,270084,1443419,1,48,65,3,6,1,3,2.331492,...,3.324514e-13,7.087074e-13,1.648484e-12,5.121786e-12,7.230008e-13,-8.446612e-13,7.961847e-16,3120,9,9.741995e-20


In [85]:
svd_dot_V = []
range_value = len(np.array(df_final_test["svd_v_d_1"]))
for i in range(range_value):
    v_s = []
    v_d = []
    v_s.append(np.array(df_final_test["svd_v_s_1"][i]))
    v_s.append(np.array(df_final_test["svd_v_s_2"][i]))
    v_s.append(np.array(df_final_test["svd_v_s_3"][i]))
    v_s.append(np.array(df_final_test["svd_v_s_4"][i]))
    v_s.append(np.array(df_final_test["svd_v_s_5"][i]))
    v_s.append(np.array(df_final_test["svd_v_s_6"][i]))

    v_d.append(np.array(df_final_test["svd_v_d_1"][i]))
    v_d.append(np.array(df_final_test["svd_v_d_2"][i]))
    v_d.append(np.array(df_final_test["svd_v_d_3"][i]))
    v_d.append(np.array(df_final_test["svd_v_d_4"][i]))
    v_d.append(np.array(df_final_test["svd_v_d_5"][i]))
    v_d.append(np.array(df_final_test["svd_v_d_6"][i]))

    svd_dot_V.append(np.dot(v_s,v_d))


In [86]:
df_final_test["svd_dot_V"] = svd_dot_V
df_final_test.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,...,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,followers_preferential_attach,followees_preferential_attach,svd_dot_U,svd_dot_V
0,848424,784690,1,6,6,14,9,1,0,0.000000,...,9.994076e-10,5.791909e-10,3.512363e-07,2.486660e-09,-2.771007e-09,1.727693e-12,36,196,8.424654e-20,2.074807e-17
1,1288618,816620,1,24,31,12,22,4,10,11.923852,...,9.083281e-10,6.860622e-10,2.349945e-11,4.769777e-10,-1.302439e-09,5.961292e-11,744,144,8.289328e-17,4.686777e-19
2,417653,942104,1,40,30,11,15,1,0,0.000000,...,1.449897e-11,2.901599e-11,1.997404e-10,4.238304e-12,-2.970304e-11,2.816912e-10,1200,121,4.247511e-20,1.799122e-20
3,971595,646855,1,8,4,4,3,2,0,0.000000,...,8.800713e-13,1.547198e-14,4.328435e-08,5.001930e-14,-2.323153e-13,1.351607e-14,32,16,1.045200e-13,2.604353e-14
4,270084,1443419,1,48,65,3,6,1,3,2.331492,...,7.087074e-13,1.648484e-12,5.121786e-12,7.230008e-13,-8.446612e-13,7.961847e-16,3120,9,9.741995e-20,1.065572e-20


In [90]:
x_train  =  df_final_train
x_test = df_final_test
print(x_train.shape)
y_train = pd.read_csv('data/train_y.csv')
print(y_train.shape)
print(x_test.shape)
y_test = pd.read_csv('data/test_y.csv')
print(y_test.shape)

(100002, 55)
(15100029, 1)
(25156, 55)
(3775007, 1)
